# Baseline Model Architecture

> Defines the baseline LLaVA 1.5 model architecture, including the projector and the combined model structure.

In [1]:
#| default_exp model.baseline

In [2]:
#| hide
from nbdev.showdoc import *

In [8]:
#| export
import sys
from pathlib import Path
import os
from typing import Any

# Assumes the notebook is run from the project root or one level down (e.g., nbs/)
# Navigate up to the project root (where settings.ini or .git likely exists)
project_root = Path(os.getcwd())
# Simple check: If settings.ini is not in cwd, assume we are in nbs/ and go up one level
if not (project_root / 'settings.ini').exists() and (project_root.parent / 'settings.ini').exists():
    project_root = project_root.parent

project_root_str = str(project_root.resolve())

if project_root_str not in sys.path:
    print(f"Adding project root to sys.path: {project_root_str}")
    sys.path.insert(0, project_root_str)
else:
    print(f"Project root already in sys.path: {project_root_str}")

Project root already in sys.path: /workspace/llava


In [4]:
#| export
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence # For padding variable length sequences
from transformers import CLIPVisionModel, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from transformers.modeling_outputs import BaseModelOutputWithPooling, CausalLMOutputWithPast # For type hints
import warnings
from typing import Optional # Added for type hinting

try:
    from peft import LoraConfig, get_peft_model, PeftModel # Import PEFT components
    _peft_available = True
except ImportError:
    print("Warning: peft library not found. LoRA functionality will be disabled.")
    LoraConfig, get_peft_model, PeftModel = None, None, None # Define as None if not available
    _peft_available = False

from llava.utils import load_config # Assuming utils notebook is created
from llava.data.preprocessing import IMAGE_TOKEN_INDEX_PLACEHOLDER, IGNORE_INDEX, tokenizer, DEFAULT_IMAGE_TOKEN # Import constants and tokenizer

# Ensure tokenizer is loaded (it should be from the import)
if tokenizer is None:
    print("Warning: Tokenizer could not be imported from data.preprocessing. Trying to load it again...")
    try:
        # Need to load config first to get LLM name
        _config_temp = load_config('configs/config.yaml')
        _llm_name_temp = _config_temp.get('model', {}).get('llm_name_or_path', 'lmsys/vicuna-7b-v1.5')
        _tokenizer_max_len_temp = _config_temp.get('data', {}).get('tokenizer_model_max_length', 2048)
        _tokenizer_padding_side_temp = _config_temp.get('data', {}).get('tokenizer_padding_side', 'right')

        from transformers import AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained(
            _llm_name_temp,
            model_max_length=_tokenizer_max_len_temp,
            padding_side=_tokenizer_padding_side_temp,
            use_fast=True,
        )
        # Re-add special token logic if necessary (should match preprocessing step)
        # from llava.data.preprocessing import DEFAULT_IMAGE_TOKEN # Already imported
        if DEFAULT_IMAGE_TOKEN not in tokenizer.get_vocab():
             num_added = tokenizer.add_special_tokens({'additional_special_tokens': [DEFAULT_IMAGE_TOKEN]})
             print(f"Added {num_added} token(s) during re-load. New vocab size: {len(tokenizer)}")
        if tokenizer.pad_token is None:
             tokenizer.pad_token = tokenizer.eos_token
             print(f"Set pad token to EOS during re-load.")
        print(f"Tokenizer re-loaded successfully for {_llm_name_temp}.")

    except Exception as e:
        print(f"Fatal Error: Could not load tokenizer. {e}")
        # Depending on the context, might raise an error or exit
        raise ImportError("Tokenizer is essential and could not be loaded.") from e

Project root already in sys.path: /workspace/llava


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Project root already in sys.path: /workspace/llava
Loaded config from configs/config.yaml
Successfully loaded CLIP image processor for: openai/clip-vit-large-patch14-336
CLIP Mean: [0.48145466, 0.4578275, 0.40821073]
CLIP Std: [0.26862954, 0.26130258, 0.27577711]
Fastai Normalize Transform: Normalize -- {'mean': tensor([[[[0.4815]],

         [[0.4578]],

         [[0.4082]]]], device='cuda:0'), 'std': tensor([[[[0.2686]],

         [[0.2613]],

         [[0.2758]]]], device='cuda:0'), 'axes': (0, 2, 3)}
(enc:2,dec:2)
Successfully loaded tokenizer for: lmsys/vicuna-7b-v1.5
Adding special token <image> to tokenizer.
Added 1 token(s). New vocab size: 32001
Using token ID for <image>: 32000
Tokenizer already has pad token: <unk> (ID: 0)


## Step 2.1: Define Projector Module

The projector module connects the vision encoder's output to the language model's input space. In LLaVA 1.5, this is typically a simple MLP.

In [5]:
#| export
class LLaVAProjector(nn.Module):
    """A simple 2-layer MLP projector to map vision features to LLM embedding space.

    Maps input_dim (e.g., CLIP ViT-L output dim = 1024) to
    output_dim (e.g., Vicuna-7B hidden dim = 4096).

    Architecture: Linear -> GELU -> Linear
    """
    def __init__(self, input_dim: int, output_dim: int):
        """Initializes the MLP projector.

        Args:
            input_dim: Dimension of the input vision features.
            output_dim: Dimension of the LLM's hidden space.
        """
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, output_dim, bias=True),
            nn.GELU(),
            nn.Linear(output_dim, output_dim, bias=True)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Projects the input features.

        Args:
            x: Input tensor of shape (batch_size, num_patches, input_dim) or similar.

        Returns:
            Projected tensor of shape (batch_size, num_patches, output_dim).
        """
        return self.model(x)

In [ ]:
show_doc(LLaVAProjector)

---

### LLaVAProjector

>      LLaVAProjector (input_dim:int, output_dim:int)

*A simple 2-layer MLP projector to map vision features to LLM embedding space.

    Maps input_dim (e.g., CLIP ViT-L output dim = 1024) to
    output_dim (e.g., Vicuna-7B hidden dim = 4096).

    Architecture: Linear -> GELU -> Linear*

In [6]:
#| test
# Test projector instantiation and forward pass
inp_dim = 1024
out_dim = 4096
projector = LLaVAProjector(input_dim=inp_dim, output_dim=out_dim)

# Create a dummy input tensor
batch_size = 2
num_patches = 576 # Example for 336x336 input with patch size 14
dummy_input = torch.randn(batch_size, num_patches, inp_dim)

# Perform forward pass
output = projector(dummy_input)

# Check output shape
assert output.shape == (batch_size, num_patches, out_dim), f"Expected shape {(batch_size, num_patches, out_dim)}, but got {output.shape}"
print("Projector test passed!")

Projector test passed!


---

## Step 2.2, 2.3, 2.4, 4.2: Define Baseline LLaVA Model & Integrate Components + LoRA

This class combines the Vision Encoder, the Projector, and the Language Model to form the complete LLaVA baseline model. We integrate the loading of the Vision Encoder (CLIP) and the Language Model (Vicuna), handle necessary configurations like embedding resizing, and **now integrate optional LoRA application using the `peft` library.**

In [9]:
#| export
class BaselineLLaVAModel(nn.Module):
    """Baseline LLaVA 1.5 model combining Vision Encoder, Projector, and LLM."""
    def __init__(self, config: dict):
        super().__init__()
        self.config = config 
        self.model_config = config.get('model', {}) 

        self.vision_tower = None
        self.language_model = None
        self.projector = None

        self.image_token_index_marker = self.model_config.get('image_token_index_marker', IMAGE_TOKEN_INDEX_PLACEHOLDER)
        self.ignore_index = IGNORE_INDEX 
        self.vision_feature_layer = self.model_config.get('vision_feature_layer', -2)
        self.vision_encoder_name = self.model_config.get('vision_encoder_name_or_path', 'openai/clip-vit-large-patch14-336')
        self.llm_name = self.model_config.get('llm_name_or_path', 'lmsys/vicuna-7b-v1.5')

        llm_hf_config = AutoConfig.from_pretrained(self.llm_name, trust_remote_code=True)
        vision_hf_config = AutoConfig.from_pretrained(self.vision_encoder_name, trust_remote_code=True)

        projector_cfg_from_yaml = self.model_config.get('projector', {})
        
        proj_input_dim_fallback = vision_hf_config.vision_config.hidden_size if hasattr(vision_hf_config, 'vision_config') else vision_hf_config.hidden_size
        proj_input_dim = projector_cfg_from_yaml.get('input_dim', proj_input_dim_fallback)
        proj_output_dim = projector_cfg_from_yaml.get('output_dim', llm_hf_config.hidden_size)

        print(f"Initializing Projector: Input Dim={proj_input_dim}, Output Dim={proj_output_dim}")
        self.projector = LLaVAProjector(proj_input_dim, proj_output_dim)

        self.load_vision_tower(expected_output_dim=proj_input_dim)
        self.load_language_model() 
        self.resize_llm_embeddings()
        self.apply_activation_checkpointing()

        if torch.cuda.is_available():
            target_device = torch.device("cuda:0")
            if self.vision_tower is not None and next(self.vision_tower.parameters()).device != target_device:
                print(f"Moving vision_tower to {target_device}")
                self.vision_tower.to(target_device)
            if self.projector is not None and next(self.projector.parameters()).device != target_device:
                print(f"Moving projector to {target_device}")
                self.projector.to(target_device)
            if self.language_model is not None:
                 llm_device = next(self.language_model.parameters()).device
                 print(f"Language model's first parameter is on device: {llm_device}")


    def load_vision_tower(self, expected_output_dim: int):
        try:
            print(f"Loading Vision Tower: {self.vision_encoder_name}...")
            self.vision_tower = CLIPVisionModel.from_pretrained(self.vision_encoder_name, trust_remote_code=True)
            print(f"Vision Tower loaded successfully.")
            self.vision_tower.requires_grad_(False)
            print(f"Vision Tower weights frozen.")
            vision_output_dim = self.vision_tower.config.hidden_size
            if vision_output_dim != expected_output_dim:
                warnings.warn(
                    f"Vision Tower output dimension ({vision_output_dim}) does not match "
                    f"Projector input dimension ({expected_output_dim})."
                )
        except Exception as e:
            print(f"Error loading Vision Tower ({self.vision_encoder_name}): {e}")
            self.vision_tower = None

    def load_language_model(self):
        try:
            print(f"Loading Language Model: {self.llm_name}...")
            
            quantization_config_dict = self.model_config.get('quantization', {})
            load_in_4bit = quantization_config_dict.get('load_in_4bit', False)
            bnb_config = None
            extra_kwargs = {"trust_remote_code": True}

            if load_in_4bit:
                if BitsAndBytesConfig is None:
                    raise ImportError("bitsandbytes library is required for 4-bit quantization but not found.")
                
                bnb_4bit_quant_type = quantization_config_dict.get('bnb_4bit_quant_type', "nf4")
                bnb_4bit_compute_dtype_str = quantization_config_dict.get('bnb_4bit_compute_dtype', "float16")
                compute_dtype = getattr(torch, bnb_4bit_compute_dtype_str)

                bnb_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_quant_type=bnb_4bit_quant_type,
                    bnb_4bit_compute_dtype=compute_dtype,
                    bnb_4bit_use_double_quant=quantization_config_dict.get('bnb_4bit_use_double_quant', False),
                )
                print(f"QLoRA enabled: Loading LLM in 4-bit with compute dtype {compute_dtype}.")
                if torch.cuda.is_available():
                    extra_kwargs["device_map"] = {"": 0} 
                    print(f"  Setting device_map to {{'': 0}} for QLoRA.")
                else:
                    print("  Warning: CUDA not available, QLoRA device_map will not be set to GPU. Model will load on CPU if possible.")
                extra_kwargs["quantization_config"] = bnb_config
            
            self.language_model = AutoModelForCausalLM.from_pretrained(
                self.llm_name,
                **extra_kwargs 
            )
            print(f"Language Model loaded successfully.")
            
            if not load_in_4bit: 
                 self.language_model.requires_grad_(False)
                 print(f"Base Language Model weights frozen (non-QLoRA path).")

            peft_config_dict = self.model_config.get('peft', {}) 
            use_lora = peft_config_dict.get('use_lora', False)

            if use_lora:
                if not _peft_available:
                    print("Warning: `use_lora` is true in config, but peft library is not installed. Skipping LoRA.")
                else:
                    if isinstance(self.language_model, PeftModel):
                        print("Language model is already a PeftModel. Ensure LoRA config matches if re-applying.")
                    
                    print("Applying LoRA...")
                    lora_r = peft_config_dict.get('lora_r', 8)
                    lora_alpha = peft_config_dict.get('lora_alpha', 16)
                    lora_dropout = peft_config_dict.get('lora_dropout', 0.05)
                    target_modules = peft_config_dict.get('target_modules', ['q_proj', 'v_proj'])
                    
                    current_lora_config = LoraConfig( 
                        r=lora_r,
                        lora_alpha=lora_alpha,
                        target_modules=target_modules,
                        lora_dropout=lora_dropout,
                        bias="none",
                    )
                    self.language_model = get_peft_model(self.language_model, current_lora_config)
                    print(f"LoRA applied. Config: r={lora_r}, alpha={lora_alpha}, dropout={lora_dropout}, modules={target_modules}")
                    self.language_model.print_trainable_parameters()
            else:
                print("LoRA is disabled in the configuration.")
        except Exception as e:
            print(f"Error loading Language Model ({self.llm_name}): {e}")
            import traceback
            traceback.print_exc()
            self.language_model = None

    def resize_llm_embeddings(self):
        if self.language_model is None or tokenizer is None:
            print("Warning: Cannot resize LLM embeddings. LLM or tokenizer not available.")
            return

        current_embeddings = self.get_input_embeddings()
        current_vocab_size = current_embeddings.weight.size(0)
        target_vocab_size = len(tokenizer)

        if current_vocab_size != target_vocab_size:
            print(f"Resizing LLM token embeddings from {current_vocab_size} to {target_vocab_size} (tokenizer size)...")
            self.language_model.resize_token_embeddings(target_vocab_size)
            print("LLM token embeddings resized.")
        else:
            print("LLM embedding size already matches tokenizer size. No resizing needed.")
            
    def apply_activation_checkpointing(self):
        use_checkpointing = self.model_config.get('use_activation_checkpointing', False)
        if use_checkpointing:
            if self.language_model is not None:
                 model_to_checkpoint = self.language_model 
                 if hasattr(model_to_checkpoint, 'gradient_checkpointing_enable'):
                     try:
                         model_to_checkpoint.gradient_checkpointing_enable()
                         print("Activation checkpointing enabled for the language model.")
                     except Exception as e:
                         print(f"Warning: Failed to enable activation checkpointing: {e}")
                 else:
                     print("Warning: Language model does not have 'gradient_checkpointing_enable' method.")
            else:
                 print("Warning: Activation checkpointing enabled in config, but language model is not loaded.")
        else:
             print("Activation checkpointing is disabled in the configuration.")

    def get_input_embeddings(self) -> nn.Embedding:
        if self.language_model is None:
            raise ValueError("Language model not loaded.")
        return self.language_model.get_input_embeddings()

    def encode_image(self, pixel_values: torch.Tensor) -> torch.Tensor | None:
        if self.vision_tower is None:
            print("Error: Vision tower not loaded, cannot encode image.")
            return None
        try:
            vision_device = next(self.vision_tower.parameters()).device
            vision_dtype = next(self.vision_tower.parameters()).dtype
            pixel_values = pixel_values.to(vision_device, dtype=vision_dtype)

            vision_outputs: BaseModelOutputWithPooling = self.vision_tower(
                pixel_values,
                output_hidden_states=True
            )
            image_features = vision_outputs.hidden_states[self.vision_feature_layer]
            image_features = image_features[:, 1:, :] 
            return image_features
        except Exception as e:
            print(f"Error during image encoding: {e}")
            import traceback
            traceback.print_exc()
            return None

    def forward(self, 
                *args: Any, # To catch positional arguments like a single dict from Learner.summary
                pixel_values: Optional[torch.Tensor] = None,
                input_ids: Optional[torch.Tensor] = None,
                attention_mask: Optional[torch.Tensor] = None, 
                labels: Optional[torch.Tensor] = None,
                **kwargs: Any
               ) -> CausalLMOutputWithPast:
        
        # Handle cases where inputs might be packed in a dictionary (e.g., from Learner)
        # or passed as keyword arguments directly.
        batch_dict = {}
        if len(args) == 1 and isinstance(args[0], dict):
            batch_dict = args[0]
        elif len(args) > 0: # Attempt to map positional args if not a dict; less robust
            # This path is less ideal; prefer keyword args or a single dict
            if pixel_values is None and isinstance(args[0], torch.Tensor): pixel_values = args[0]
            if input_ids is None and len(args) > 1 and isinstance(args[1], torch.Tensor): input_ids = args[1]
            if attention_mask is None and len(args) > 2 and (args[2] is None or isinstance(args[2], torch.Tensor)): attention_mask = args[2]
            if labels is None and len(args) > 3 and (args[3] is None or isinstance(args[3], torch.Tensor)): labels = args[3]

        # Update from kwargs if they were provided
        if 'pixel_values' in kwargs: pixel_values = kwargs['pixel_values']
        if 'input_ids' in kwargs: input_ids = kwargs['input_ids']
        if 'attention_mask' in kwargs: attention_mask = kwargs['attention_mask']
        if 'labels' in kwargs: labels = kwargs['labels']
        
        # If batch_dict was populated, use its values preferentially or if others are None
        if batch_dict:
            pixel_values = batch_dict.get('pixel_values', pixel_values)
            input_ids = batch_dict.get('input_ids', input_ids)
            attention_mask = batch_dict.get('attention_mask', attention_mask)
            labels = batch_dict.get('labels', labels)

        if pixel_values is None or input_ids is None:
            # This error will be caught by the Learner.summary() or training loop if inputs are malformed.
            raise ValueError("forward() missing required arguments: pixel_values and input_ids must be provided.")

        if self.language_model is None or self.vision_tower is None or self.projector is None:
            raise RuntimeError("Model components (LLM, Vision Tower, Projector) are not fully loaded.")

        image_features = self.encode_image(pixel_values)
        if image_features is None:
            raise RuntimeError("Image encoding failed.")
        
        projector_device = next(self.projector.parameters()).device
        image_features = image_features.to(projector_device)
        projected_image_features = self.projector(image_features)
        num_image_patches = projected_image_features.shape[1]
        
        embedding_layer = self.get_input_embeddings()
        target_device = embedding_layer.weight.device
        
        input_ids_clone = input_ids.clone().to(target_device) 
        input_ids_clone[input_ids_clone == self.image_token_index_marker] = 0 
        
        text_embeddings = embedding_layer(input_ids_clone)
        projected_image_features = projected_image_features.to(target_device, dtype=text_embeddings.dtype)

        new_input_embeds = []
        new_labels = [] if labels is not None else None
        new_attention_mask = []

        for batch_idx in range(input_ids.shape[0]):
            current_input_ids_slice = input_ids[batch_idx].to(target_device)
            image_token_indices = torch.where(current_input_ids_slice == self.image_token_index_marker)[0]
            
            if len(image_token_indices) == 0:
                warnings.warn(f"Image token placeholder {self.image_token_index_marker} not found in batch index {batch_idx}. Skipping image features.")
                new_input_embeds.append(text_embeddings[batch_idx])
                current_attention_mask_slice = attention_mask[batch_idx].to(target_device) if attention_mask is not None else (current_input_ids_slice != tokenizer.pad_token_id).long()
                new_attention_mask.append(current_attention_mask_slice)
                if new_labels is not None and labels is not None:
                    new_labels.append(labels[batch_idx].to(target_device))
                continue

            image_token_start_index = image_token_indices[0].item()
            text_emb_before = text_embeddings[batch_idx, :image_token_start_index]
            text_emb_after = text_embeddings[batch_idx, image_token_start_index + 1:]

            cur_new_embed = torch.cat([
                text_emb_before,
                projected_image_features[batch_idx], 
                text_emb_after
            ], dim=0)
            new_input_embeds.append(cur_new_embed)

            current_attention_mask_slice = attention_mask[batch_idx].to(target_device) if attention_mask is not None else (current_input_ids_slice != tokenizer.pad_token_id).long()
            mask_before = current_attention_mask_slice[:image_token_start_index]
            mask_image = torch.ones(num_image_patches, dtype=torch.long, device=target_device)
            mask_after = current_attention_mask_slice[image_token_start_index + 1:]
            cur_new_mask = torch.cat([mask_before, mask_image, mask_after], dim=0)
            new_attention_mask.append(cur_new_mask)

            if new_labels is not None and labels is not None:
                current_labels_slice = labels[batch_idx].to(target_device)
                label_before = current_labels_slice[:image_token_start_index]
                label_image = torch.full((num_image_patches,), self.ignore_index, dtype=torch.long, device=target_device)
                label_after = current_labels_slice[image_token_start_index + 1:]
                cur_new_label = torch.cat([label_before, label_image, label_after], dim=0)
                new_labels.append(cur_new_label)

        padded_input_embeds = pad_sequence(new_input_embeds, batch_first=True, padding_value=0.0)
        padded_attention_mask = pad_sequence(new_attention_mask, batch_first=True, padding_value=0)
        padded_labels = None
        if new_labels is not None:
            padded_labels = pad_sequence(new_labels, batch_first=True, padding_value=self.ignore_index)
        
        outputs: CausalLMOutputWithPast = self.language_model(
            inputs_embeds=padded_input_embeds, 
            attention_mask=padded_attention_mask, 
            labels=padded_labels, 
            return_dict=True
        )
        return outputs

In [ ]:
show_doc(BaselineLLaVAModel, name='BaselineLLaVAModel')

---

### BaselineLLaVAModel

>      BaselineLLaVAModel (config:dict)

*Baseline LLaVA 1.5 model combining Vision Encoder, Projector, and LLM.

    Loads the specified Vision Encoder (CLIP) and LLM (Vicuna) from Hugging Face Hub,
    along with the Projector. Handles freezing components, LLM embedding resizing,
    and optionally applies LoRA to the LLM based on the configuration.
    Implements `encode_image`, activation checkpointing, and the main `forward` pass.*

In [8]:
#| test
# Test BaselineLLaVAModel instantiation including LLM loading and embedding resize
import torch, gc # Add torch and gc
from transformers import AutoModelForCausalLM, CLIPVisionModel # Add imports

try:
    config_path = '../configs/config.yaml'
    config = load_config(config_path)
    print(f"Loaded config from {config_path}")
    
    # --- Test with LoRA enabled (assuming config enables it) --- 
    config['model']['peft']['use_lora'] = True # Ensure LoRA is tested
    config['model']['use_activation_checkpointing'] = False # Disable checkpointing for this basic test
    
    baseline_model = BaselineLLaVAModel(config)

    # Basic checks
    assert isinstance(baseline_model, nn.Module)
    assert baseline_model.projector is not None
    assert baseline_model.vision_tower is not None
    assert isinstance(baseline_model.vision_tower, CLIPVisionModel)
    assert baseline_model.language_model is not None # Should be loaded now
    # Check if LLM is wrapped by PeftModel if LoRA is enabled
    if _peft_available and config['model']['peft']['use_lora']:
         assert isinstance(baseline_model.language_model, PeftModel), f"Expected PeftModel, but got {type(baseline_model.language_model)}"
         print("Language model correctly wrapped by PeftModel.")
    else:
         assert isinstance(baseline_model.language_model, AutoModelForCausalLM)
         print("Language model is standard AutoModelForCausalLM (LoRA disabled or peft not available).")
         
    print("BaselineLLaVAModel test passed!")
    print(f"  - Projector exists: {baseline_model.projector is not None} (Type: {type(baseline_model.projector)})")
    print(f"  - Vision Tower exists: {baseline_model.vision_tower is not None} (Type: {type(baseline_model.vision_tower)})")
    print(f"  - Language Model exists: {baseline_model.language_model is not None} (Type: {type(baseline_model.language_model)})")

    # Test encode_image helper (already tested, but good to re-check)
    print("\nTesting encode_image...")
    batch_size = 2
    dummy_pixel_values = torch.randn(batch_size, 3, 336, 336) # Example image batch
    image_features = baseline_model.encode_image(dummy_pixel_values)

    assert image_features is not None
    expected_shape = (batch_size, 576, 1024)
    assert image_features.shape == expected_shape, f"Expected shape {expected_shape}, but got {image_features.shape}"
    print(f"Image features shape: {image_features.shape}")
    print("encode_image test passed!")

    # Test embedding size
    print("\nTesting LLM embedding size...")
    llm_embeddings = baseline_model.get_input_embeddings()
    llm_vocab_size = llm_embeddings.weight.size(0)
    tokenizer_vocab_size = len(tokenizer)
    print(f"LLM embedding vocab size: {llm_vocab_size}")
    print(f"Tokenizer vocab size: {tokenizer_vocab_size}")
    assert llm_vocab_size == tokenizer_vocab_size, \
        f"Mismatch! LLM embedding size ({llm_vocab_size}) != Tokenizer size ({tokenizer_vocab_size})"
    print("LLM embedding size test passed!")

except FileNotFoundError:
    print(f"Config file {config_path} not found. Skipping BaselineLLaVAModel test.")
except ImportError as e:
    print(f"Skipping test due to ImportError: {e}. (Likely `peft` is missing)")
except Exception as e:
    print(f"An error occurred during BaselineLLaVAModel test: {e}")
    import traceback
    traceback.print_exc()

# Clean up large model from memory if running tests in a notebook
# import gc # Already imported
if 'baseline_model' in locals():
    # Ensure model is moved to CPU before deleting if it was on GPU
    if hasattr(baseline_model, 'vision_tower') and baseline_model.vision_tower is not None:
        baseline_model.vision_tower.to('cpu')
    if hasattr(baseline_model, 'language_model') and baseline_model.language_model is not None:
        baseline_model.language_model.to('cpu')
    if hasattr(baseline_model, 'projector') and baseline_model.projector is not None:
        baseline_model.projector.to('cpu')
    del baseline_model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Cleaned up baseline_model")

Loaded config from ../configs/config.yaml
Initializing Projector: Input Dim=1024, Output Dim=4096
Loading Vision Tower: openai/clip-vit-large-patch14-336...
Vision Tower loaded successfully.
Vision Tower weights frozen.
Loading Language Model: lmsys/vicuna-7b-v1.5...
QLoRA enabled: Loading LLM in 4-bit with compute dtype torch.float16.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/venv/main/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/transformers/generation/configuratio

Language Model loaded successfully.
Base Language Model weights frozen.
Applying LoRA...


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


LoRA applied. Config: r=8, alpha=16, dropout=0.05, modules=['q_proj', 'v_proj']
trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622
Resizing LLM token embeddings from 32000 to 32001 (tokenizer size)...
LLM token embeddings resized.
Activation checkpointing is disabled in the configuration.
Language model correctly wrapped by PeftModel.
BaselineLLaVAModel test passed!
  - Projector exists: True (Type: <class '__main__.LLaVAProjector'>)
  - Vision Tower exists: True (Type: <class 'transformers.models.clip.modeling_clip.CLIPVisionModel'>)
  - Language Model exists: True (Type: <class 'peft.peft_model.PeftModel'>)

Testing encode_image...
Image features shape: torch.Size([2, 576, 1024])
encode_image test passed!

Testing LLM embedding size...
LLM embedding vocab size: 32001
Tokenizer vocab size: 32001
LLM embedding size test passed!
Cleaned up baseline_model


---

### Forward Pass Test

In [10]:
#| test
# Test the implemented forward pass
import torch, gc
from transformers.modeling_outputs import CausalLMOutputWithPast

try:
    config_path = '../configs/config.yaml'
    config = load_config(config_path)
    print(f"Loaded config from {config_path}")
    
    # --- Configure for QLoRA and LoRA for this test ---
    if 'model' not in config: config['model'] = {}
    if 'peft' not in config['model']: config['model']['peft'] = {}
    if 'quantization' not in config['model']: config['model']['quantization'] = {}
    
    config['model']['quantization']['load_in_4bit'] = True # Enable QLoRA
    config['model']['quantization']['bnb_4bit_compute_dtype'] = "float16" # Ensure a compatible dtype
    config['model']['peft']['use_lora'] = True # QLoRA implies LoRA
    config['model']['use_activation_checkpointing'] = False # Keep disabled for this basic test

    # Instantiate the model
    forward_test_model = BaselineLLaVAModel(config)
    forward_test_model.eval() # Set model to evaluation mode

    # Determine the device of the model
    # (assuming all parts of the model are on the same device after initialization,
    # which device_map="auto" or BitsAndBytesConfig usually handles for quantized models)
    model_device = next(forward_test_model.parameters()).device
    print(f"Model is on device: {model_device}")

    print("Running forward pass test...")
    # Prepare dummy inputs
    batch_size = 2
    seq_len = 15 
    img_size = 336 
    num_patches = 576 
    llm_hidden_dim = config['model']['projector']['output_dim']
    
    # Ensure tokenizer is loaded and get vocab size
    if tokenizer is None:
        raise RuntimeError("Tokenizer is not loaded. Cannot proceed with test.")
    tokenizer_vocab_size = len(tokenizer)

    # Move dummy tensors to the model's device
    dummy_pixel_values = torch.randn(batch_size, 3, img_size, img_size, device=model_device)
    dummy_input_ids = torch.randint(1, tokenizer_vocab_size, (batch_size, seq_len), dtype=torch.long, device=model_device)
    placeholder_idx = 5 
    dummy_input_ids[:, placeholder_idx] = IMAGE_TOKEN_INDEX_PLACEHOLDER # This value is int, device doesn't apply until it's in a tensor
    
    dummy_attention_mask = torch.ones_like(dummy_input_ids, device=model_device)
    dummy_labels = dummy_input_ids.clone() # Will also be on model_device
    dummy_labels[:, :placeholder_idx+1] = IGNORE_INDEX 

    print("Created dummy inputs and moved to model device.")
    print(f"Original input_ids shape: {dummy_input_ids.shape}, device: {dummy_input_ids.device}")

    # Perform forward pass
    with torch.no_grad(): 
         outputs = forward_test_model(
            pixel_values=dummy_pixel_values,
            input_ids=dummy_input_ids,
            attention_mask=dummy_attention_mask,
            labels=dummy_labels
         )

    # Check output type and attributes
    assert isinstance(outputs, CausalLMOutputWithPast)
    assert hasattr(outputs, 'logits')
    assert outputs.logits is not None
    assert hasattr(outputs, 'loss') 
    assert outputs.loss is not None

    # Check output shapes
    expected_seq_len = seq_len - 1 + num_patches
    expected_logits_shape = (batch_size, expected_seq_len, tokenizer_vocab_size)

    print(f"Padded inputs_embeds shape (expected): ({batch_size}, {expected_seq_len}, {llm_hidden_dim})")
    print(f"Padded attention_mask shape (expected): ({batch_size}, {expected_seq_len})")
    print(f"Padded labels shape (expected): ({batch_size}, {expected_seq_len})")

    assert outputs.logits.shape == expected_logits_shape, \
        f"Expected logits shape {expected_logits_shape}, but got {outputs.logits.shape}"
    print(f"Output logits shape: {outputs.logits.shape}")
    print(f"Output loss: {outputs.loss}")
    print("Forward pass test successful!")

except FileNotFoundError:
    print(f"Config file {config_path} not found. Skipping forward pass test.")
except ImportError as e:
     print(f"Skipping forward pass test due to ImportError: {e}. (Likely `peft` or `bitsandbytes` is missing)")
except Exception as e:
    print(f"An error occurred during forward pass test: {e}")
    import traceback
    traceback.print_exc()

# Clean up
if 'forward_test_model' in locals():
    # Models with device_map="auto" or BitsAndBytes quantization might not need explicit .to('cpu')
    # as their components can be on different devices or offloaded.
    # Simply deleting the reference should be sufficient for GC.
    del forward_test_model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Cleaned up forward_test_model")

Loaded config from ../configs/config.yaml
Initializing Projector: Input Dim=1024, Output Dim=4096
Loading Vision Tower: openai/clip-vit-large-patch14-336...
Vision Tower loaded successfully.
Vision Tower weights frozen.
Loading Language Model: lmsys/vicuna-7b-v1.5...
QLoRA enabled: Loading LLM in 4-bit with compute dtype torch.float16.
  Setting device_map to {'': 0} for QLoRA.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/venv/main/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/transformers/generation/configuratio

Language Model loaded successfully.
Applying LoRA...
LoRA applied. Config: r=8, alpha=16, dropout=0.05, modules=['q_proj', 'v_proj']
trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622
Resizing LLM token embeddings from 32000 to 32001 (tokenizer size)...


The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


LLM token embeddings resized.
Activation checkpointing is disabled in the configuration.
Moving vision_tower to cuda:0
Moving projector to cuda:0
Language model's first parameter is on device: cuda:0
Model is on device: cuda:0
Running forward pass test...
Created dummy inputs and moved to model device.
Original input_ids shape: torch.Size([2, 15]), device: cuda:0
Padded inputs_embeds shape (expected): (2, 590, 4096)
Padded attention_mask shape (expected): (2, 590)
Padded labels shape (expected): (2, 590)
Output logits shape: torch.Size([2, 590, 32001])
Output loss: 11.768798828125
Forward pass test successful!
Cleaned up forward_test_model


In [12]:
#| hide
import nbdev; nbdev.nbdev_export()